In [1]:

from __future__ import annotations
from datetime import datetime, timezone
from syft_event import SyftEvents

from syft_core import Client
from syft_rds.models.dataset import Dataset, DatasetCreate
from syft_rds.service.dataset_service import DatasetService
from syft_rds.connection.connection import get_connection
from syft_rds.service.context import BaseRPCContext

ModuleNotFoundError: No module named 'anthropic'

In [2]:
client = Client.load()

: 

In [ ]:

box = SyftEvents("my-rds-app")

# TODO: pass this in the handler
context = BaseRPCContext(client=client, box=box)  
        

@box.on_request("/datasets/create")
def create_dataset(dataset: DatasetCreate) -> Dataset:
    """Respond to a ping request."""
    dataset_service = DatasetService.from_context(context)
    res = dataset_service.create_item(dataset)
    return res
    


: 

In [4]:
conn = get_connection(box, mock=True)

: 

In [ ]:
# DatasetCreate(name="my-dataset", description="my dataset").to_item()

: 

In [ ]:
res = conn.send(
    url=f"syft://{client.email}/api_data/my-rds-app/rpc/datasets/create",
    body=DatasetCreate(name="my-dataset", description="my dataset"),
    expiry="5m",
    cache=True,
)

: 

In [ ]:
%debug

: 

: 